In [ ]:
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
import theano
import theano.tensor as T
import scipy
from scipy import misc
import skimage
from skimage.transform import resize
from skimage.io import imread
from theano_model import vgg19_model, preliminary_model
reload(vgg19_model)
VGG19 = vgg19_model.VGG19

import  NeuralStyle


In [ ]:
image_h = 124
image_w = 124

In [ ]:
content_path = 'images/tubingen.jpg'
style_path = 'images/starry_night.jpg'
nst = NeuralStyle.NeuralStyleTransfer(content_path, style_path,image_w,image_h)
prep_img_l = nst.prep_img_l



In [ ]:
cont = imread('images/potrait1.jpg')
style = imread('images/picasso.jpg')
raw_content, content = prep_img_l(cont)
raw_style, style = prep_img_l(style)
white_noise = np.random.uniform(low=-128.0, high=128.0, size=(1, 3, image_w, image_h)).astype(theano.config.floatX)
#white_noise=content

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize = (10,5))
ax1.imshow(raw_content)
ax2.imshow(raw_style)

In [ ]:
wn = theano.shared(value=white_noise, name='wn', borrow=False)
vgg19 = VGG19(input_image_shape=(1,3,image_w,image_h), pool_method = 'average_exc_pad')
#vgg19 = VGG19(input_image_shape=(224,224,3))


get_content_layer = theano.function(inputs=[vgg19.input],
                                    outputs=vgg19.conv4_2.output)

get_style_layers = theano.function(inputs=[vgg19.input],
                                    outputs=[vgg19.conv3_1.output, vgg19.conv4_1.output])

sty_out = get_style_layers(style)
cont_lay = get_content_layer(content)

sty_out_white = get_style_layers(wn.get_value())
cont_lay_white = get_content_layer(wn.get_value())

In [ ]:
from theano.tensor.nnet import conv2d
from theano.tensor.nnet.neighbours import images2neibs

In [ ]:
sty_out = get_style_layers(style)
cont_lay = get_content_layer(content)

sty_out_white = get_style_layers(wn.get_value())

In [ ]:
########################################################
white_neib_input = vgg19.conv3_1.output
style_neib_input = theano.tensor.tensor4('style_neib_input')

n_shape = 5
n_step = 5

white_neib = images2neibs(ten4 = white_neib_input,
                    neib_shape = (n_shape, n_shape),
                    neib_step = (n_step, n_step),
                    mode = 'ignore_borders')


style_neib = images2neibs(ten4 = style_neib_input,
                    neib_shape = (n_shape, n_shape),
                    neib_step = (n_step, n_step),
                    mode = 'ignore_borders')



mat = theano.dot(white_neib, style_neib.T)
white_norms = white_neib / white_neib.sum(axis=1).reshape((white_neib.shape[0], 1))
style_norms = style_neib / style_neib.sum(axis=1).reshape((style_neib.shape[0], 1))


denominator = (white_norms*style_neib).sum(axis = 1)
mat2 = mat / denominator

closest_matches = np.argmax(mat2, axis=0)
style_energy = (white_neib - style_neib[closest_matches]).norm(L = 2,axis=None)




########################################################
white_neib1_input = vgg19.conv4_1.output
style_neib1_input = theano.tensor.tensor4('style_neib1_input')




white_neib1 = images2neibs(ten4 = white_neib1_input,
                    neib_shape = (n_shape, n_shape),
                    neib_step = (n_step, n_step),
                    mode = 'ignore_borders')


style_neib1 = images2neibs(ten4 = style_neib1_input,
                    neib_shape = (n_shape, n_shape),
                    neib_step = (n_step, n_step),
                    mode = 'ignore_borders')




mat1 = theano.dot(white_neib1, style_neib1.T)
white_norms1 = white_neib1 / white_neib1.sum(axis=1).reshape((white_neib1.shape[0], 1))
style_norms1 = style_neib1 / style_neib1.sum(axis=1).reshape((style_neib1.shape[0], 1))


denominator1 = (white_norms1*style_neib1).sum(axis = 1)
mat21 = mat1 / denominator1

closest_matches1 = np.argmax(mat21, axis=0)
style_energy1 = (white_neib1 - style_neib1[closest_matches1]).norm(L = 2,axis=None)


style_cost = style_energy + style_energy1


cont = T.dtensor4('cont')

cont_loss = 0.5*T.sum(T.sqr(vgg19.conv4_2.output-cont))


total_cost = style_cost + cont_loss
img_grad = T.grad(total_cost, vgg19.input)

In [ ]:
f_loss = theano.function([vgg19.input,cont,style_neib_input,style_neib1_input], total_cost)
f_grad = theano.function([vgg19.input,cont,style_neib_input,style_neib1_input],img_grad)

def lbfgs_loss(x0):
    x0 = x0.reshape((1, 3, image_w,image_h)).astype(theano.config.floatX)
    wn.set_value(x0)
    loss = f_loss(wn.get_value(), cont_lay,sty_out[0],sty_out[1]).astype('float64')
    print loss
    return loss

def lbfgs_gradient(x0):
    x0 = x0.reshape((1, 3, image_w,image_h)).astype(theano.config.floatX)
    wn.set_value(x0)
    gradient = np.array(f_grad(wn.get_value(), cont_lay,sty_out[0],sty_out[1])).flatten().astype('float64')
    return gradient

In [ ]:

x0 = wn.get_value().astype('float64')
xs = []
xs.append(x0)
cont_lay = get_content_layer(content)
sty_out = get_style_layers(style)

for i in range(9):
    print(i)
    scipy.optimize.fmin_l_bfgs_b(lbfgs_loss, x0.flatten(), fprime=lbfgs_gradient, maxfun=20)
    x0 = wn.get_value().astype('float64')
#     joblib.dump(x0,'results/art_test' + str(i))
    xs.append(x0)

In [ ]:
plt.imshow(np.transpose(x0[0],(1,2,0)))